# Tensorflow API 中的tf.nn，tf.layers与tf.contrib

我们在使用tensorflow时，会发现tf.nn，tf.layers， tf.contrib模块有很多功能是重复的，尤其是卷积操作。

在使用的时候，我们可以根据需使用不同的模块也可以一起混用。

## tf.nn
tf.nn ：提供神经网络相关操作的支持，包括卷积操作（conv）、池化操作（pooling）、归一化、loss、分类操作、embedding、RNN、Evaluation。

## tf.layers
定义在tensorflow/python/layers/layers.py, 为我们提供了一些高层次的构建神经网络的接口。
今后会被放到keras模块中

## tf.contrib
tf.contrib.layers提供够将计算图中的 网络层、正则化、摘要操作、是构建计算图的高级操作，但是tf.contrib包含不稳定和实验代码。

tF.Contrib，开源社区贡献，新功能，内外部测试，根据反馈意见改进性能，改善API友好度，API稳定后，移到TensorFlow核心模块。

<img src="./images/05_01.png"> <br>

## 以conv2d举例
让我们看一下tf.nn.conv2d和tf.layers.conv2d之间的显着差异。

后者处理权重和偏差所需的所有变量，单行代码。

<img src="./images/05_02.png"> <br>

对于上一节的实例中，使用tf.nn.conv2d创建卷积层，必须在将权重变量传递给函数之前自己声明权重变量。
然后自己需要添加对应激活函数。
<img src="./images/05_03.png"> <br>
除此之外，tf.layers.conv2d还建议在同一个函数调用中添加正则化和激活，可以想象当更高级别的API覆盖了需求时，这可以减少代码大小.


# 让我们使用高级API进行tensorflow模型的构建

## 首先使用keras

In [2]:
import tensorflow as tf
train, test = tf.keras.datasets.mnist.load_data()
mnist_x, mnist_y = train

11493376/11490434 [==============================] - 2s 0us/step


In [3]:
mnist_x.shape

(60000, 28, 28)

In [4]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
"""
现在我们可以开始实现第一层了。
它由一个卷积接一个max pooling完成。
卷积在每个5x5的patch中算出32个特征。
卷积的权重张量形状是[5, 5, 1, 32]，前两个维度是patch的大小，接着是输入的通道数目，最后是输出的通道数目。
而对于每一个输出通道都有一个对应的偏置量。
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
"""
# Layer 1
model.add(layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='same', input_shape=(28, 28, 1)))
model.add(layers.MaxPool2D(pool_size=(2, 2), padding='same', strides=(2, 2)))

"""
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
"""
          
# Layer 2
model.add(layers.Conv2D(filters=64, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='same'))
model.add(layers.MaxPool2D(pool_size=(2, 2), padding='same', strides=(2, 2)))

"""
现在，图片尺寸减小到7x7，我们加入一个有1024个神经元的全连接层，用于处理整个图片。
我们把池化层输出的张量reshape成一些向量，乘上权重矩阵，加上偏置，然后对其使用ReLU。
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
"""
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              3212288   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total para

`tf.keras.Model.compile` 采用三个重要参数：

`optimizer`：此对象会指定训练过程。从 `tf.train` 模块向其传递优化器实例，例如 `tf.train.AdamOptimizer`、`tf.train.RMSPropOptimizer` 或 `tf.train.GradientDescentOptimizer`。
`loss`：要在优化期间最小化的函数。常见选择包括均方误差 (mse)、categorical_crossentropy 和 binary_crossentropy。损失函数由名称或通过从 `tf.keras.losses` 模块传递可调用对象来指定。
`metrics`：用于监控训练。它们是 tf.keras.metrics 模块中的字符串名称或可调用对象。

In [6]:
model.compile(optimizer=tf.train.AdamOptimizer(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [7]:
model.fit(mnist_x.reshape(60000, 28, 28, 1), tf.keras.utils.to_categorical(mnist_y, 10), epochs=5, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 37s 620us/sample - loss: 3.4827 - acc: 0.7586
Epoch 2/5
60000/60000 [==============================] - 6s 100us/sample - loss: 1.7814 - acc: 0.8740
Epoch 3/5
60000/60000 [==============================] - 6s 99us/sample - loss: 1.4871 - acc: 0.9006
Epoch 4/5
60000/60000 [==============================] - 6s 100us/sample - loss: 1.4714 - acc: 0.9050
Epoch 5/5
60000/60000 [==============================] - 6s 99us/sample - loss: 1.4650 - acc: 0.9070


## 首先tf.layers中的层来构建我们的模型

In [8]:
import tensorflow as tf
train, test = tf.keras.datasets.mnist.load_data()
mnist_x, mnist_y = train

In [9]:
from tensorflow.keras import layers

In [10]:
x_image = tf.reshape(x, [-1,28,28,1])
"""
现在我们可以开始实现第一层了。
它由一个卷积接一个max pooling完成。
卷积在每个5x5的patch中算出32个特征。
卷积的权重张量形状是[5, 5, 1, 32]，前两个维度是patch的大小，接着是输入的通道数目，最后是输出的通道数目。
而对于每一个输出通道都有一个对应的偏置量。
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
"""
# Layer 1
conv1 = layers.conv2d(inputs=x_image, filters=32, kernel_sie=(5, 5), activation='relu', padding='same')
pool1 = layers.max_pooling2d(inputs=conv1, pool_size=(2, 2), padding='same', strides=(2, 2))

"""
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
"""
          
# Layer 2
conv2 = layers.conv2d(inputs=pool1, filters=64, kernel_sie=(5, 5), activation='relu', padding='same')
pool2 = layers.max_pooling2d(inputs=conv2, pool_size=(2, 2), padding='same', strides=(2, 2))

"""
现在，图片尺寸减小到7x7，我们加入一个有1024个神经元的全连接层，用于处理整个图片。
我们把池化层输出的张量reshape成一些向量，乘上权重矩阵，加上偏置，然后对其使用ReLU。
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
"""
flatten = layers.flatten(inputs=pool2, name='flatten')
fc1 = layers.dense(inputs=flatten, units=1024, activation='relu')
fc2 = layers.dense(inputs=fc2, units=10, activation='softmax')

NameError: name 'x' is not defined